# Text Data

Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

In [ ]:
from requests import get
import re

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px

import nltk

from nltk.corpus import stopwords
from nltk import SnowballStemmer, LancasterStemmer
from nltk.tokenize import RegexpTokenizer

import string

## Processing Text Data

Recall that we were able to use the [NYT API](https://developer.nytimes.com/apis) to pull data from New York Times articles, including the abstract from each of the articles. Suppose we wanted to know what types of articles were being published. We can look at the information from the abstracts and read through them, but this would take a very long time since there are so many articles. Instead, we can use **text analysis** to help us.

However, as-is, the text from abstracts can be difficult to analyze. We aren't able to use traditional statistical techniques without some heavy data manipulation, because the text is essentially a categorical variable with unique values for each patent. We need to basically break it apart and clean the data before we apply our data analysis techniques. 

Here, we will go through the process of cleaning and processing the text data, as well as visualizing and exploring the text data that we have. 

## Processing Text Data

**Text analysis** is used to extract useful information from or summarize a large amount of unstructured text stored in documents. This opens up the opportunity of using text data alongside more conventional data sources (e.g. surveys and administrative data). The goal of text analysis is to take a large corpus of complex and unstructured text data and extract important and meaningful messages in a comprehensible way. 

Text analysis can help with a wide variety of tasks including:

* **Information Retrieval**: Find relevant information in a large database, such as a systematic literature review, that would be very time-consuming for humans to do manually. 

* **Clustering and Text Categorization**: Summarize a large corpus of text by finding the most important phrases, using methods like topic modeling. 

* **Text Summarization**: Create category-sensitive text summaries of a large corpus of text. 

* **Machine Translation**: Translate documents from one language to another. 
 
## Glossary of Terms

* **Corpus**: A corpus is the set of all text documents used in your analysis; for example, your corpus of text may include hundreds of abstracts from patent data.

* **Tokenize**: Tokenization is the process by which text is separated into meaningful terms or phrases. In English this is easy to do for individual words, as they are separated by whitespace; however, it can get more complicated to  automate determining which groups of words constitute meaningful phrases. 

* **Stemming**: Stemming is normalizing text by reducing all forms or conjugations of a word to the word's most basic form. In English, this can mean making a rule of removing the suffixes "ed" or "ing" from the end of all words, but it gets more complex. For example, "to go" is irregular, so you need to tell the algorithm that "went" and "goes" stem from a common lemma, and should be considered alternate forms of the word "go."

* **Stop Words**: Stop words are words that have little semantic meaning but occur very frequently, like prepositions, articles and common nouns. For example, every document (in English) will probably contain the words "and" and "the" many times. You will often remove them as part of preprocessing using a list of stop words.


In [ ]:
with open('nyt-key.txt', 'r') as f:
    nyt_key = f.readline()

In [ ]:
base_url = "https://api.nytimes.com/svc/archive/v1/2019/1.json"

To pull from the API, we need to provide the API key. For this API, we can use the parameter of `api-key` with our key.

In [ ]:
r = get(base_url, params= {'api-key':nyt_key})  

We are good to go. Now let's get the content.

In [ ]:
articles = r.json()['response']['docs']
articles[0]

In [ ]:
keys = ['web_url','abstract', 'pub_date', 'type_of_material','word_count']

nyt_dict = {key:[article[key] for article in articles] for key in keys}

In [ ]:
nyt_df = pd.DataFrame(nyt_dict)

In [ ]:
nyt_df.head()

First, we will lowercase every word in the abstract column so that we don't run into issues with matching words that aren't capitalized in the same way. 

In [ ]:
nyt_df.abstract = nyt_df.abstract.str.lower()

In [ ]:
nyt_df.head()

## Preparing Text Data for Natural Language Processing (NLP)

The first important step in working with text data is cleaning and processing the data, which includes (but is not limited to):

- forming a corpus of text
- stemming and lemmatization
- tokenization
- removing stop-words

The ultimate goal is to transform our text data into a form an algorithm can work with, because a document or a corpus of text cannot be fed directly into an algorithm. Algorithms expect numerical feature vectors with certain fixed sizes, and can't handle documents, which are basically sequences of symbols with variable length. We will be transforming our text corpus into a *bag of n-grams* to be further analyzed.

### Removing Punctuation and Tokenizing

For some purposes, we might want to preserve punctuation. For example, if we wanted to be able to detect sentiment of text, we might want to keep exclamation points, because they signify something about the text. For our purposes, however, we will simply strip the punctuation so that it does not affect our analysis. 

We also want to separate text into individual tokens (generally individual words). To do this all of this, we'll use `RegexpTokenizer` to break apart individual words and identify tokens according to a regular expression. Here, we use the regular expression `\w+`, which catches all "words" which contain letters or numbers. Anything that is not a number, such as punctuation or spaces, will not be included and be considered a separator between tokens.

In [ ]:
# Before
nyt_df.abstract[0]

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer

Let's take a look at an example of how this works using the first abstract in our corpus.

In [ ]:
tokenizer.tokenize(abstracts[0])

<font color ='red'>**Question 1: Apply the tokenizer to each abstract in `nyt_df`. You should get a pandas Series of lists, with each list representing the tokenized abstract. Call this Series `bag_of_words`.**</font>


What we get out of it is something called a **bag of words**. This is a list of all of the words that are in the abstract, cleaned of all punctuation and stemmed. The paragraph is now represented as a vector of individual words rather than as one whole entity. 

In [ ]:
words_and_type = pd.DataFrame({'abstract': bag_of_words, 
                               'type_of_material': nyt_df.type_of_material}).explode('abstract')
words_and_type.head()

### Stemming and Lemmatization - Distilling text data

We want to process our text through *stemming and lemmatization*, or replacing words with their root or simplest form. For example "systems," "systematic," and "system" are all different words, but we can replace all these words with "system" without sacrificing much meaning. 

- A **lemma** is the original dictionary form of a word (e.g. the lemma for "lies," "lied," and "lying" is "lie").
- The process of turning a word into its simplest form is **stemming**. There are several well known stemming algorithms -- Porter, Snowball, Lancaster -- that all have their respective strengths and weaknesses.

In this notebook, we'll use the Snowball Stemmer:

In [ ]:
# Examples of how a Stemmer works:
stemmer = SnowballStemmer("english")
print(stemmer.stem('lies'))
print(stemmer.stem("lying"))
print(stemmer.stem('systematic'))
print(stemmer.stem("running"))

In [ ]:
words_and_type.abstract = words_and_type.abstract.apply(stemmer.stem)
words_and_type.head()

<font color ='red'>**Question 2: Try stemming using an alternative stemmer: the Lancaster stemmer. How does it differ from the Snowball Stemmer?**</font>


In [ ]:
lancaster = LancasterStemmer()




### Visualizing the Data

Let's take a look at our data to see what types of words are most common. This is a common intermediate step that is used to simply get a sense for what is in our dataset. Remember, we are converting blocks of text into individual words to try to get a sense for what these articles are about. So, a natural first step is to just see which word are the most common. This should hopefully give us an idea of what types of concepts are being discussed the most in New York Times articles.

In [ ]:
top_ten_words = words_and_type.abstract.value_counts().head(10)
px.bar(top_ten_words[::-1], orientation = 'h')

### Removing meaningless text - Stopwords

Stopwords are words that are found commonly throughout a text and carry little semantic meaning. Examples of common stopwords are prepositions ("to", "on", "in"), articles ("the", "an", "a"), conjunctions ("and", "or", "but") and common nouns. For example, the words *the* and *of* are totally ubiquitous, so they won't serve as meaningful features, whether to distinguish documents from each other or to tell what a given document is about. You may also run into words that you want to remove based on where you obtained your corpus of text or what it's about. There are many lists of common stopwords available for you to use, both for general documents and for specific contexts, so you don't have to start from scratch.   

We can eliminate stopwords by checking all the words in our corpus against a list of commonly occuring stopwords that comes with NLTK.

In [ ]:
# Download most current stopwords
nltk.download('stopwords')

In [ ]:
stop = stopwords.words('english')
stop[0:10]

We have the stopwords we want to remove in `stop`. So, we first need to stem the stop words.

<font color ='red'>**Question 3: Create a list called `eng_stopwords` that contains the stop words in `stop` stemmed.**</font>


To remove stop words, we use the `isin` method to check whether words are in the list of stop words. Then, we take any that are not in the list of stop words.

In [ ]:
cleaned_words_and_type = words_and_type[-words_and_type.abstract.isin(eng_stopwords)]

Now, let's visualize the words again to see if we get something a bit more meaningful.

<font color ='red'>**Question 4: Create a bar graph of the frequency of the top ten most frequent words in our corpus after having removed stop words. What are the most frequent words?**</font>


### Words by Type

What if we were interested in the words used by type of article. For example, are different words used in Op-Eds compared to News articles? We can check this by looking at the distribution of articles for each of the top words. 

In [ ]:
words_by_type = pd.crosstab(cleaned_words_and_type.abstract, cleaned_words_and_type.type_of_material)

Let's keep only the most frequent words so that we don't have too many to look at at once.

In [ ]:
top_words_by_type = words_by_type.loc[top_words.index[::-1],:]
top_words_by_type

In [ ]:
px.bar(top_words_by_type, 
       x = top_words_by_type.columns, y = top_words_by_type.index, 
       orientation = 'h')

## Next Steps

There's much more that is possible with text data. We'll be looking at a few of these over the next few weeks.

- **More visualizations and exploration**: We only looked at one month's worth of data. Typically, data like this are studied over longer periods of time. What are the trends like over a year? How would you study this?
- **Topic modeling**: We've only looked overall at all of the words together. However, these words come from individual articles, or documents. What types of documents are there? What are the topics of those documents? 
- **Sentiment Analysis**: We might be interested in the types of sentiment expressed in text. Is the document positive or negative? We can use sentiment analysis techniques to determine this.
- **AI and text prediction**: How can we use text data with AI and neural networks to answer some of these questions and more? 